In [ ]:
!pip install docling

In [ ]:
from docling.document_converter import DocumentConverter

source_file = "files/Resolucao_4.pdf"
converter = DocumentConverter()
result = converter.convert(source_file)

doc = result.document
markdown_content = doc.export_to_markdown()

with open("file.md", "w", encoding="utf-8") as file:
    file.write(markdown_content)

In [ ]:
!pip install -qU "langchain-chroma>=0.1.2"
!pip install -qU langchain-openai
!pip install -qU langchain_community unstructured
!pip install -qU langchain-text-splitters
!pip install -qU langchain-core
!pip install "unstructured[md]"

In [6]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [9]:
from langchain_chroma import Chroma

In [10]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [14]:
def load_documents(file_path):
    
    print(f"Carregando documento: {file_path}")
    
    loader = UnstructuredMarkdownLoader(
        file_path,
        mode="single",
        strategy="fast",
    )

    documents = loader.load()
    
    print(f"Documento carregado com {len(documents[0].page_content)} caracteres")
    
    return documents

docs = load_documents("./data/leis/Lei_organica.md")

Carregando documento: ./data/leis/Lei_organica.md
Documento carregado com 162578 caracteres


In [15]:
def split_documents(documents):
    text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1200,
            chunk_overlap=150,
            separators=[
                "\nArt. ",
                "\nArtigo ",
                "\n§",
                "\nI -", "\nII -", "\nIII -",
                "\n\n",
                "\n",
                ". ",
                " "
            ]
        )
    
    chunks = text_splitter.split_documents(documents)
    print(f"Documento dividido em {len(chunks)} chunks")
    
    print(f"\n Exemplo do primeiro chunk:")
    print(f"{chunks[0].page_content[:100]}")
    
    return chunks

text_chunks = split_documents(docs)

Documento dividido em 229 chunks

 Exemplo do primeiro chunk:
LEI ORGÂNICA

LEI ORGÂNICA DO MUNICÍPIO DE CAÇADOR/SC.

Nós, representantes do povo caçadorense, com


In [16]:
def create_vectorstore(chunks, persist_directory="./chroma_db"):
    """Cria vector store com ChromaDB"""
    
    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-small"
    )

    # Criar vectorstore vazia
    vectorstore = Chroma(
        collection_name="lei_organica",  # Nome fixo como string
        embedding_function=embeddings,
        persist_directory=persist_directory
    )
    
    # Adicionar documentos
    vectorstore.add_documents(chunks)
    
    print(f"Vector store criado com {len(chunks)} documentos")
    
    return vectorstore

vectorstore = create_vectorstore(text_chunks)

print(vectorstore._collection.count())

2025-12-15 14:54:13,314 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-12-15 14:54:17,217 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Vector store criado com 229 documentos


In [17]:
def setup_qa_chain(vectorstore):
    """Configuração da chain"""

    # Prompt
    template = """Você é um assistente especializado nas Leis do Município de Caçador/SC.
    Use o contexto abaixo para responder a pergunta.

    REGRAS:
        - Cite artigos específicos quando relevante (ex: "Conforme art. 23...")
        - Se não souber a resposta com base no contexto, diga que não sabe.
        - Utilize citações das leis quando possível.
        - Utilize até 30 linhas para a resposta.
        - Seja claro e direto, respondendo de forma concisa.

    Contexto:
    {context}

    Pergunta:
    {question}

    Resposta detalhada:"""

    prompt = PromptTemplate.from_template(template)

    # LLM
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0
    )

    # Retriever
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": 5}
    )

    # Chain substituindo o RetrievalQA repassado pelo professor
    qa_chain = (
        {
            "context": retriever,
            "question": RunnablePassthrough()
        }
        | prompt
        | llm
    )

    print("Chain configurada com sucesso!")
    return qa_chain


qa_chain = setup_qa_chain(vectorstore)

Chain configurada com sucesso!


In [26]:
def ask_question(chain, question):
    print(f"\n {question}\n")
    
    response = chain.invoke(question)
    answer = response.content if hasattr(response, 'content') else str(response)
    
    print(f"{answer}")
    
    return

# Exemplos

In [27]:
ask_question(qa_chain, "Qual é a Subseção VI Das Comissões?")


 Qual é a Subseção VI Das Comissões?



2025-12-15 15:07:49,939 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-12-15 15:07:52,972 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


A Subseção VI Das Comissões, conforme o contexto fornecido, foi revogada pela Emenda à Lei Orgânica nº 15/2015. Portanto, não há disposições ou artigos vigentes que tratem sobre essa subseção no atual texto da Lei Orgânica do Município de Caçador/SC. 

Se precisar de informações sobre outras subseções ou artigos que ainda estão em vigor, estou à disposição para ajudar!


In [20]:
ask_question(qa_chain, "Quais são os poderes do município?")


 Quais são os poderes do município?



2025-12-15 15:05:07,231 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-12-15 15:05:15,342 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Os poderes do município de Caçador/SC estão delineados na sua Lei Orgânica, que estabelece diversas atribuições e competências. Abaixo, destaco algumas das principais atribuições do município:

1. **Legislar sobre assuntos de interesse local** (art. 23, I).
2. **Suplementar a legislação federal e estadual** quando necessário (art. 23, III).
3. **Elaborar o Plano Plurianual, a Lei de Diretrizes Orçamentárias e o Orçamento Anual** (art. 23, IV).
4. **Instituir e arrecadar tributos municipais**, aplicando suas rendas e prestando contas (art. 23, V).
5. **Criar, organizar e suprimir distritos**, conforme a legislação estadual e municipal (art. 23, VII).
6. **Dispor sobre a administração e utilização dos bens públicos** (art. 23, VIII).
7. **Organizar e prestar serviços públicos locais**, incluindo transporte coletivo (art. 23, XI).
8. **Decretar desapropriações** por necessidade ou utilidade pública (art. 23, VII).
9. **Expedir decretos e atos administrativos** para a execução das leis (ar

In [21]:
ask_question(qa_chain, "Qual o percentual mínimo que o município deve aplicar em educação?")


 Qual o percentual mínimo que o município deve aplicar em educação?



2025-12-15 15:05:32,635 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-12-15 15:05:36,286 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Conforme o art. 172 da Lei Orgânica do Município de Caçador/SC, "O Município aplicará anualmente 25% (vinte e cinco por cento), no mínimo, da receita resultante de impostos, compreendida a proveniente de transferências, na manutenção e desenvolvimento do ensino". Portanto, o percentual mínimo que o município deve aplicar em educação é de 25% da sua receita proveniente de impostos.

